In [ ]:
import itertools
from pprint import pprint
import copy

gp.parisizemax=512000000000000;

def valuation(P,I):
     F = list(I.factor());
     for i in range(0,len(F)):
         if not F[i][0].is_coprime(P):
             return F[i][1];
     return 0;

def group_by_conjugates(primes,taus):
     already_viewed = [];
     conj_pairs = [];
     for i in range(0,len(primes)):
         if i not in already_viewed:
             p = primes[i];
             already_viewed.append(i);
             found = False;
             for j in range(i+1,len(primes)):
                 q = primes[j].apply_morphism(taus[1]);
                 if (j not in already_viewed) and (not q.is_coprime(p)) and (not found):
                     found = True;
                     already_viewed.append(j);
                     conj_pairs.append([p,primes[j]]);
             if not found:
                 conj_pairs.append([p,p]);
     return conj_pairs;

#Construction of the Equation x^2 + D = 5y^n
[cstart,dstart, lstart, sgn] = [5,11,4*(99),-1]; #Y^2 = cstart X^(2dstart) + sgn*lstart (include 4 in here)
[C,D, n] = [cstart,-sgn*lstart,dstart] #x^2 + D = Cy^n 
print("Eqn: x^2 +",D,"=",C,"y^",n)
print("(c,d,l,+/-) = (",cstart,dstart,lstart,sgn,")")


d = squarefree_part(D);
q = sqrt(abs(D/d));

#Establish the Number Field K = Q(sqrt(-d))
var('x')
#branch off here?
K.<a>=NumberField(x^2+d);
taus=K.embeddings(K);
O = K.maximal_order(); #ring of integers
H, h = K.class_group(), K.class_number();
U = UnitGroup(K);
#u = U.gens();
U_reps = [];
for u in U.gens():
     i=0;
     while i<n:
         if u**i not in U_reps:
             U_reps.append(u**i);
             U_reps.append(U.gens()[0]*u**i); ##u0 is minus one
         i += 1;
print("Representatives of U/U^n:",U_reps) #figuring out all possible u value
#print()



discK = K.discriminant();
w = K.ideal(1).basis()[1]; # K is spanned by 1 and 1 # ideal generated by 1 in K
wbar = taus[1]((K.ideal(1).basis()[1]));#ideal is generated by two elements 


TwoDC =2*D*C; #changed for arb C
TwoDC_ideal = K.ideal(TwoDC);
S_2DC = list(TwoDC_ideal.prime_factors());
print("Primes dividing <2DC>:")
for x in S_2DC: print(x)
print()


conj_primes = group_by_conjugates(S_2DC,taus); # takes prime factors of <2DC> and then 
print("list of the primes", conj_primes);

F= list(factor(TwoDC));
F1 = sorted(set([f[0] for f in F for _ in range(f[1])])); #doesn't need to be sorted 
print("primes dividing 2DC",F1)
f=len(F1);

#print(conj_primes)

#-----------------------------------------------------------------------------------------
## Since 2CD = 10D, the prime factors are 2, 5, D
Sall = [];
for i in range(f):
 #   setname = "S"+str(i);
    setname = [];
    for k in (0..n-1):
        for l in (0..n-1): ##kp= k and Kp bar = l
            if min(k,l)<=valuation(conj_primes[i][0],K.ideal(4*q*a)) and mod(k+l-valuation(conj_primes[i][0],K.ideal(C)),n)==0:
                if conj_primes[i][0].is_coprime(conj_primes[i][1]): setname.append(conj_primes[i][0]^(k)*conj_primes[i][1]^(l));
                elif k==l: setname.append(conj_primes[i][0]^(k));
    Sall.append(setname)
#    if not setname == []: 
#        Sall.append(setname);
if [] in Sall:
    Sall.remove([])
print("Sall",Sall)


a_plus_ideals = [];

if len(Sall) == 1:
    for p in list(itertools.product(*[Sall[0]])):
        a_plus_ideals.append(p)
if len(Sall) == 2:
    for p,k in list(itertools.product(*[Sall[0],Sall[1]])):
        a_plus_ideals.append(p*k)
if len(Sall) == 3:
    for p,k,r in list(itertools.product(*[Sall[0],Sall[1],Sall[2]])):
        a_plus_ideals.append(p*k*r)
if len(Sall) == 4:
    for p,k,r in list(itertools.product(*[Sall[0],Sall[1],Sall[2]])):
        a_plus_ideals.append(p*k*r)

#for p in list(itertools.product(*Sall)):
   # a_plus_ideals.append(prod(p)
    
print("aplus",a_plus_ideals)

#------------------------------------------------------------------------------------------
H_elts = [i.ideal() for i in list(H)];
#H_elts = [H.gen().ideal()^i for i in (1..3) ]
gammas = [1];
for a_ideal in a_plus_ideals:
     for g in H_elts:
         ideal = a_ideal*g^(-n);
         if ideal.is_principal():
             gammas.append(ideal.gens_reduced()[0]);

Gamma = [];
for g in gammas:
     for u in U_reps:
         Gamma.append([taus[0](K(u)*g), taus[1](K(u)*g)]);
#print()
#print(Gamma)
print()


R.<X,Y> = K['X','Y'];

thues = [];
for r in Gamma:
     thues.append(1/a*(r[0]*(X+Y*w)**n-r[1]*(X+Y*wbar)**n)); ##equation 2.6 produce
        
# print(thues)
SOLNS = [];
#for t in thues: print(t) #solving for (A,B) in 2.6
#the two cases merge back?

count = 0;
for r,f in zip(Gamma,thues):
     print(count+1,"of",len(thues))
     count = count + 1;
     fnew = simplify(f.denominator()*expand(f(Y=1)));
     th=gp.thueinit(fnew,0); #grh assuming 0; not grh assuming is 1
     #print(2*q*f.denominator(),th[1])
     th_solns = gp.thue(th,2*q*f.denominator());
     #print(th_solns)
     #print()
     for s in th_solns:
         SOLNS.append([s,r]);
#print()
#print('SOLUTION',SOLNS)

################################################################

#taking AB and morphing into equation for x
FINAL_SOLNS = [];
for e in SOLNS:
     x = 1/2 * (e[1][0]*(pari(e[0][1])+pari(e[0][2])*w)**n +
e[1][1]*(pari(e[0][1])+pari(e[0][2])*wbar)**n);
     FINAL_SOLNS.append(x);
print("Final Solutions",FINAL_SOLNS)




Eqn: x^2 + 396 = 5 y^ 11
(c,d,l,+/-) = ( 5 11 396 -1 )
Representatives of U/U^n: [1, u]
Primes dividing <2DC>:
Fractional ideal (2)
Fractional ideal (1/2*a - 1/2)
Fractional ideal (-1/2*a - 1/2)
Fractional ideal (1/2*a + 3/2)
Fractional ideal (-1/2*a + 3/2)
Fractional ideal (-a)

list of the primes [[Fractional ideal (2), Fractional ideal (2)], [Fractional ideal (1/2*a - 1/2), Fractional ideal (-1/2*a - 1/2)], [Fractional ideal (1/2*a + 3/2), Fractional ideal (-1/2*a + 3/2)], [Fractional ideal (-a), Fractional ideal (-a)]]
primes dividing 2DC [2, 3, 5, 11]
Sall [[Fractional ideal (1)], [Fractional ideal (1), Fractional ideal (111*a - 204), Fractional ideal (-111*a - 204)], [Fractional ideal (-1/2*a + 3/2), Fractional ideal (1/2*a + 3/2)], [Fractional ideal (1)]]
aplus [Fractional ideal (-1/2*a + 3/2), Fractional ideal (1/2*a + 3/2), Fractional ideal (537/2*a + 609/2), Fractional ideal (129/2*a - 1833/2), Fractional ideal (-129/2*a - 1833/2), Fractional ideal (-537/2*a + 609/2)]

1 of 1